# Purpose

The code herein is used to pull together the chargemaster dataset

In [1]:
import pandas as pd
import requests

In [7]:
metadata = pd.read_excel('chargemasters/chargemaster_index.xlsx', sheet_name = 'Sheet1')
metadata

,Document ID,Hospital,URL,File Format,Notes,Secondary URL (e.g. common landing page)
0,0,Atlanticare Regional Medical Center,https://www.atlanticare.org/assets/images/serv...,CSV,NaN,NaN
1,1,Aurora BayCare Medical Center,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
2,2,Aurora Medical Center in Burlington,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
3,3,Aurora Medical Center in Grafton,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
4,4,Aurora Medical Center in Kenosha,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
5,5,Aurora Lakeland Medical Center,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
6,6,Aurora Medical Center in Manitowoc County,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
7,7,Aurora Medical Center in Oshkosh,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
8,8,Aurora Psychiatric Hospital,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN
9,9,Aurora Sheboygan Memorial Medical Center,https://www.aurorahealthcare.org/-/media/auror...,CSV,NaN,NaN


In [109]:
import openpyxl
import pandas as pd

def get_URLs(cell_range, filepath = 'chargemasters/chargemaster_index.xlsx', sheet_name = 'Sheet1'):
    '''
    Extract embedded URLs from cells in an Excel workbook.
    
    Parameters
    ----------
    filepath: str. Relative (to working directory) filepath of workbook
    
    sheet_name: str. Name of worksheet in your workbook that contains the cells of interest
    
    cell_range: two-element tuple of str. Defines the first and last cell in a column 
        from which you want to extract embedded URLs
        
    Returns
    -------
    urls: pandas Series of str. Each element is an embedded URL. If a URL can't be found, element is None.
        Index of urls should match corresponding DataFrame row index for relevant subset of data if XLSX
        were imported as a DataFrame using read_excel(). Assumes that: Excel row number = (DataFrame index + 1)
    '''
    
    wb = openpyxl.load_workbook(filepath)
    ws = wb[sheet_name]
    cells = ws[cell_range[0]:cell_range[1]]
    index = [cell.row - 1 for e in cells for cell in e]
    
    urls = [cell.hyperlink.target if cell.hyperlink is not None else None for e in cells for cell in e]
    
    return pd.Series(urls, index = index)


In [110]:
get_URLs(('C833', 'C850'))

832    https://oshpd.ca.gov/ml/v1/resources/document?...
833    https://oshpd.ca.gov/ml/v1/resources/document?...
834    https://oshpd.ca.gov/ml/v1/resources/document?...
835    https://oshpd.ca.gov/ml/v1/resources/document?...
836    https://oshpd.ca.gov/ml/v1/resources/document?...
837    https://oshpd.ca.gov/ml/v1/resources/document?...
838    https://oshpd.ca.gov/ml/v1/resources/document?...
839    https://oshpd.ca.gov/ml/v1/resources/document?...
840    https://oshpd.ca.gov/ml/v1/resources/document?...
841    https://oshpd.ca.gov/ml/v1/resources/document?...
842    https://oshpd.ca.gov/ml/v1/resources/document?...
843    https://oshpd.ca.gov/ml/v1/resources/document?...
844    https://oshpd.ca.gov/ml/v1/resources/document?...
845    https://oshpd.ca.gov/ml/v1/resources/document?...
846    https://oshpd.ca.gov/ml/v1/resources/document?...
847                                                 None
848                                                 None
849                            

In [ ]:
#TODO: now that we have a list of links we can generate, figure out how to reliably download the files